## Let's try GCN and MLP first

In [30]:
addRawFeat = True
base_path = ''
feature_networks_integration = ['clinical', 'cna', 'exp','coe','met','mut'] # datatypes to concatanate node features from
node_networks = ['clinical', 'cna', 'exp','coe','met','mut'] # datatypes to use networks from
int_method = 'MLP' # Machine Learning method to integrate node embeddings: 'MLP' or 'XGBoost' or 'RF' or 'SVM'

# optimize for hyperparameter tuning
learning_rates = [0.01, 0.001, 0.0001] # learning rates to tune for GCN
hid_sizes = [32, 64, 128, 256] # hidden sizes to tune for GCN
xtimes = 50 #number of times Machine Learning algorithm will be tuned for each combination
xtimes2 = 3 # number of times each evaluation metric will be repeated (for standard deviation of evaluation metrics)

# optimize for optional feature selection of node features
feature_selection_per_network = [False, False, False,False, False, False]
top_features_per_network = [50, 50, 50,50,50,50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50


# fixed
max_epochs = 500
min_epochs = 200
patience = 30

# fixed to get the same results from the tool each time
random_state = 404

# SUPREME run
print('SUPREME is setting up!')
from lib import module
import time
import os, itertools
import pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import os
import torch
import argparse
from tqdm import tqdm
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

if ((True in feature_selection_per_network) or (optional_feat_selection == True)):
    import rpy2
    import rpy2.robjects as robjects
    from rpy2.robjects.packages import importr
    utils = importr('utils')
    rFerns = importr('rFerns')
    Boruta = importr('Boruta')
    pracma = importr('pracma')
    dplyr = importr('dplyr')
    import re

dataset_name = 'full_data'

path = base_path + "data/" + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cuda:1')

node_networks = ['mut'] # datatypes to use networks from
modelname = 'MLP'

SUPREME is setting up!


In [31]:

def train():
    model.train()
    optimizer.zero_grad()
    out, emb1 = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return emb1


def validate():
    model.eval()
    with torch.no_grad():
        out, emb2 = model(data)
        pred = out.argmax(dim=1)
        loss = criterion(out[data.valid_mask], data.y[data.valid_mask])        
    return loss, emb2

def test():
    model.eval()
    with torch.no_grad():
        out, emb2 = model(data)
        pred = out.argmax(dim=1)
        y_pred = pred[data.test_mask].cpu()
        return accuracy_score(y_pred, y_test)
        
        

criterion = torch.nn.CrossEntropyLoss()

data_path_node =  base_path + 'data/' + dataset_name +'/'
run_name = 'SUPREME_'+  dataset_name + '_results'
save_path = base_path + run_name + '/'

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

file = base_path + 'data/' + dataset_name +'/labels.pkl'
with open(file, 'rb') as f:
    labels = pickle.load(f)

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
    print('use pre-defined split')
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels)
    print('use random split')
start = time.time()


is_first = 0

print('SUPREME is running..')
# Node feature generation - Concatenating node features from all the input datatypes            
for netw in node_networks:
    file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    with open(file, 'rb') as f:
        feat = pickle.load(f)
        if feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] < feat.values.shape[1]:     
            feat_flat = [item for sublist in feat.values.tolist() for item in sublist]
            feat_temp = robjects.FloatVector(feat_flat)
            robjects.globalenv['feat_matrix'] = robjects.r('matrix')(feat_temp)
            robjects.globalenv['feat_x'] = robjects.IntVector(feat.shape)
            robjects.globalenv['labels_vector'] = robjects.IntVector(labels.tolist())
            robjects.globalenv['top'] = top_features_per_network[node_networks.index(netw)]
            robjects.globalenv['maxBorutaRuns'] = boruta_runs
            robjects.r('''
                require(rFerns)
                require(Boruta)
                labels_vector = as.factor(labels_vector)
                feat_matrix <- Reshape(feat_matrix, feat_x[1])
                feat_data = data.frame(feat_matrix)
                colnames(feat_data) <- 1:feat_x[2]
                feat_data <- feat_data %>%
                    mutate('Labels' = labels_vector)
                boruta.train <- Boruta(feat_data$Labels ~ ., data= feat_data, doTrace = 0, getImp=getImpFerns, holdHistory = T, maxRuns = maxBorutaRuns)
                thr = sort(attStats(boruta.train)$medianImp, decreasing = T)[top]
                boruta_signif = rownames(attStats(boruta.train)[attStats(boruta.train)$medianImp >= thr,])
                    ''')
            boruta_signif = robjects.globalenv['boruta_signif']
            robjects.r.rm("feat_matrix")
            robjects.r.rm("labels_vector")
            robjects.r.rm("feat_data")
            robjects.r.rm("boruta_signif")
            robjects.r.rm("thr")
            topx = []
            for index in boruta_signif:
                t_index=re.sub("`","",index)
                topx.append((np.array(feat.values).T)[int(t_index)-1])
            topx = np.array(topx)
            values = torch.tensor(topx.T, device=device)
        elif feature_selection_per_network[node_networks.index(netw)] and top_features_per_network[node_networks.index(netw)] >= feat.values.shape[1]:
            values = feat.values
        else:
            values = feat.values
    
    if is_first == 0:
        new_x = torch.tensor(values, device=device).float()
        is_first = 1
    else:
        new_x = torch.cat((new_x, torch.tensor(values, device=device).float()), dim=1)

        

# Node embedding generation using GCN for each input network with hyperparameter tuning   
for n in range(len(node_networks)):
    netw_base = node_networks[n]
    with open(data_path_node + 'edges_' + netw_base + '.pkl', 'rb') as f:
        edge_index = pickle.load(f)
    best_ValidLoss = np.Inf
    learning_rate = 0.001
    hid_size = 128
    av_valid_losses = list()

    data = Data(x=new_x, edge_index=torch.tensor(edge_index[edge_index.columns[0:2]].transpose().values, device=device).long(),
                edge_attr=torch.tensor(edge_index[edge_index.columns[2]].transpose().values, device=device).float(), y=labels) 

    X = data.x[train_valid_idx]
    y = data.y[train_valid_idx]
    rskf = RepeatedStratifiedKFold(n_splits=4, n_repeats=1)

    for train_part, valid_part in rskf.split(X, y):
        train_idx = train_valid_idx[train_part]
        valid_idx = train_valid_idx[valid_part]
        break

    train_mask = np.array([i in set(train_idx) for i in range(data.x.shape[0])])
    valid_mask = np.array([i in set(valid_idx) for i in range(data.x.shape[0])])
    data.valid_mask = torch.tensor(valid_mask, device=device)
    data.train_mask = torch.tensor(train_mask, device=device)
    test_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
    data.test_mask = torch.tensor(test_mask, device=device)
    y_test = pd.DataFrame(data.y[data.test_mask].cpu().numpy()).values.ravel()

    in_size = data.x.shape[1]
    out_size = torch.unique(data.y).shape[0]
    if modelname=='GCN':
        model = module.GCN(in_size=in_size, hid_size=hid_size, out_size=out_size)
    elif modelname=='MLP':
        model = module.MLP(in_size=in_size, hid_size=hid_size, out_size=out_size)
        
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)
    data = data.to(device)
    min_valid_loss = np.Inf
    patience_count = 0

    for epoch in tqdm(range(max_epochs)):
        emb = train()
        this_valid_loss, emb = validate()

        if this_valid_loss < min_valid_loss:
            min_valid_loss = this_valid_loss
            patience_count = 0
            this_emb = emb
        else:
            patience_count += 1

        if epoch >= min_epochs and patience_count >= patience:
            break

    av_valid_losses.append(min_valid_loss.item())

    av_valid_loss = round(statistics.median(av_valid_losses), 3)

    if av_valid_loss < best_ValidLoss:
        best_ValidLoss = av_valid_loss
        best_emb_lr = learning_rate
        best_emb_hs = hid_size
        selected_emb = this_emb
        test_acc = test()
        print('network ',n,'test acc',test_acc)

 

use pre-defined split
SUPREME is running..


 40%|████      | 200/500 [00:00<00:00, 538.26it/s]

network  0 test acc 0.42391304347826086


GCN:
    network  0 test acc 0.7971014492753623
    network  1 test acc 0.7753623188405797
    network  2 test acc 0.7934782608695652

MLP:
    network  0 test acc 0.782608695652174
    network  1 test acc 0.7862318840579711
    network  2 test acc 0.7681159420289855